In [35]:
import time
import datetime
import os
import ast
import sys
import pytz
import random
import shutil
# import splitfolders
import copy
from glob import glob
from tqdm import tqdm
from pprint import pprint

import warnings
import requests

import gc
import math
import numpy as np
import pandas as pd

import cv2
from PIL import Image
# import pydicom as pdm
import pickle
import yaml
import h5py
import json
import matplotlib.pyplot as plt
# import seaborn as sns

import albumentations as albu
import torch
import torch.nn as nn  
# import torchtuples as tt
from sklearn import metrics

import torchvision
import timm 
# import pretrainedmodels

In [213]:
with open('predicted_labels_lstm_v005.npy', 'rb') as f:
    predicted_labels = np.load(f, allow_pickle = True)
with open('predicted_values_lstm_v005.npy', 'rb') as f:
    predicted_values = np.load(f, allow_pickle = True)

In [214]:
len(predicted_labels)

32891

In [215]:
# with open('predicted_labels_max_v008.npy', 'rb') as f:
#     predicted_labels = np.load(f, allow_pickle = True)

In [216]:
# with open('predicted_values_max_v008.npy', 'rb') as f:
#     predicted_values = np.load(f, allow_pickle = True)

In [217]:
predicted_values[700:]

array([tensor([ 0.3205, -0.1709, -0.0966], device='cuda:0'),
       tensor([ 0.3205, -0.1709, -0.0966], device='cuda:0'),
       tensor([ 0.3205, -0.1709, -0.0966], device='cuda:0'), ...,
       tensor([ 0.2133,  0.2165, -0.2205], device='cuda:0'),
       tensor([ 0.2133,  0.2165, -0.2205], device='cuda:0'),
       tensor([ 0.2133,  0.2165, -0.2205], device='cuda:0')], dtype=object)

In [218]:
predicted_labels = list(predicted_labels)
len(predicted_labels)

32891

In [219]:
# predicted_labels = [predicted_labels.tolist()[i].tolist()[0][0] for i in range(len(predicted_labels))]

In [220]:
set(predicted_labels),\
predicted_labels.count(0),\
predicted_labels.count(1),\
predicted_labels.count(2)

({0, 1, 2}, 7976, 23448, 1467)

In [221]:
X = pd.read_csv('test_csv.csv')

In [222]:
len(X)

32888

In [228]:
X['answer'] = predicted_labels[:-3]
X['Epoch'] = np.array(X['Epoch'])+1
X

,Case,Epoch,Epoch_Count,Frame_Count,Frame_Rate,Frame,ID,answer
0,A2020-EM-01-0005,1,923,138163,4.995,"[0, 149]",0,2
1,A2020-EM-01-0005,2,923,138163,4.995,"[149, 299]",1,2
2,A2020-EM-01-0005,3,923,138163,4.995,"[299, 449]",2,2
3,A2020-EM-01-0005,4,923,138163,4.995,"[449, 599]",3,2
4,A2020-EM-01-0005,5,923,138163,4.995,"[599, 749]",4,2
...,...,...,...,...,...,...,...,...
32883,A2021-EM-01-0163,930,935,139323,4.972,"[138569, 138718]",32883,1
32884,A2021-EM-01-0163,931,935,139323,4.972,"[138718, 138867]",32884,1
32885,A2021-EM-01-0163,932,935,139323,4.972,"[138867, 139017]",32885,1
32886,A2021-EM-01-0163,933,935,139323,4.972,"[139017, 139166]",32886,1


In [229]:
formatt = pd.read_csv('formatt.csv')

In [230]:
cases = []
for case in list(X['Case']):
    if case not in cases:
        print(case)
        print(list(X[X['Case'] == case]['Epoch'])[-1])
    cases.append(case)

A2020-EM-01-0005
922
A2020-EM-01-0006
856
A2020-EM-01-0020
803
A2020-EM-01-0028
765
A2020-EM-01-0081
855
A2020-EM-01-0085
781
A2020-EM-01-0090
933
A2020-EM-01-0099
834
A2020-EM-01-0110
748
A2020-EM-01-0113
786
A2020-EM-01-0117
881
A2020-EM-01-0119
775
A2020-EM-01-0182
957
A2020-EM-01-0185
830
A2020-EM-01-0186
782
A2020-EM-01-0187
838
A2021-EM-01-0005
694
A2021-EM-01-0011
748
A2021-EM-01-0014
954
A2021-EM-01-0015
764
A2021-EM-01-0021
845
A2021-EM-01-0025
832
A2021-EM-01-0028
858
A2021-EM-01-0047
788
A2021-EM-01-0053
853
A2021-EM-01-0057
839
A2021-EM-01-0058
820
A2021-EM-01-0078
852
A2021-EM-01-0082
954
A2021-EM-01-0087
715
A2021-EM-01-0101
868
A2021-EM-01-0102
657
A2021-EM-01-0110
727
A2021-EM-01-0111
870
A2021-EM-01-0112
813
A2021-EM-01-0120
728
A2021-EM-01-0127
757
A2021-EM-01-0129
838
A2021-EM-01-0162
834
A2021-EM-01-0163
934


In [231]:
X = X[['Case', 'Epoch','answer']]

In [232]:
X = X.rename(columns={'Case': 'Video_name', 'Epoch':'epoch'})
X

,Video_name,epoch,answer
0,A2020-EM-01-0005,1,2
1,A2020-EM-01-0005,2,2
2,A2020-EM-01-0005,3,2
3,A2020-EM-01-0005,4,2
4,A2020-EM-01-0005,5,2
...,...,...,...
32883,A2021-EM-01-0163,930,1
32884,A2021-EM-01-0163,931,1
32885,A2021-EM-01-0163,932,1
32886,A2021-EM-01-0163,933,1


In [233]:
result = pd.merge(X, formatt, on=['Video_name', 'epoch'], how = 'outer')

In [234]:
result = result[['Video_name', 'epoch', 'answer_x']]
result = result.rename(columns={'answer_x':'answer'})

In [235]:
result = result.fillna(method='ffill')

In [236]:
result['answer'] = pd.to_numeric(result['answer'], downcast='integer')

In [237]:
result['answer'].tolist()

[2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [238]:
result = result.replace({"answer": {0:"Deep Sleep", 1:"Light Sleep", 2:"Wake"}})
result

,Video_name,epoch,answer
0,A2020-EM-01-0005,1,Wake
1,A2020-EM-01-0005,2,Wake
2,A2020-EM-01-0005,3,Wake
3,A2020-EM-01-0005,4,Wake
4,A2020-EM-01-0005,5,Wake
...,...,...,...
32886,A2021-EM-01-0163,933,Light Sleep
32887,A2021-EM-01-0163,934,Light Sleep
32888,A2020-EM-01-0113,787,Light Sleep
32889,A2021-EM-01-0078,853,Light Sleep


In [239]:
answer_lst = result['answer'].tolist()
for idx in range(len(answer_lst)):
    if result['epoch'].tolist()[idx] in [1,2,3,4,5]:
        answer_lst[idx] = "Wake"

answer_lst[:10]

['Wake',
 'Wake',
 'Wake',
 'Wake',
 'Wake',
 'Light Sleep',
 'Light Sleep',
 'Light Sleep',
 'Light Sleep',
 'Light Sleep']

In [240]:
answer_lst.count("Deep Sleep"),\
answer_lst.count("Light Sleep"),\
answer_lst.count("Wake")

(7971, 23345, 1575)

In [241]:
result['answer'] = answer_lst

In [242]:
result = result.sort_values(by=['Video_name', 'epoch'])

In [243]:
list(formatt[['Video_name']]) == list(result[['Video_name']]),\
list(formatt[['epoch']]) == list(result[['epoch']])

(True, True)

In [244]:
result.to_csv('submittttt.csv', index = False)

In [166]:
!ls /ainode/dataset/train/rgb/A2020-EM-01-0001_video_01

img_00000.jpg	img_114391.jpg	img_22511.jpg  img_48342.jpg  img_74173.jpg
img_00001.jpg	img_114392.jpg	img_22512.jpg  img_48343.jpg  img_74174.jpg
img_00002.jpg	img_114393.jpg	img_22513.jpg  img_48344.jpg  img_74175.jpg
img_00003.jpg	img_114394.jpg	img_22514.jpg  img_48345.jpg  img_74176.jpg
img_00004.jpg	img_114395.jpg	img_22515.jpg  img_48346.jpg  img_74177.jpg
img_00005.jpg	img_114396.jpg	img_22516.jpg  img_48347.jpg  img_74178.jpg
img_00006.jpg	img_114397.jpg	img_22517.jpg  img_48348.jpg  img_74179.jpg
img_00007.jpg	img_114398.jpg	img_22518.jpg  img_48349.jpg  img_74180.jpg
img_00008.jpg	img_114399.jpg	img_22519.jpg  img_48350.jpg  img_74181.jpg
img_00009.jpg	img_11440.jpg	img_22520.jpg  img_48351.jpg  img_74182.jpg
img_00010.jpg	img_114400.jpg	img_22521.jpg  img_48352.jpg  img_74183.jpg
img_00011.jpg	img_114401.jpg	img_22522.jpg  img_48353.jpg  img_74184.jpg
img_00012.jpg	img_114402.jpg	img_22523.jpg  img_48354.jpg  img_74185.jpg
img_00013.jpg	img_114403.jpg	img_22524.jpg  img_4835

In [195]:
import shutil

In [248]:
!ls ./submission/submitted/

21-11-01_00:39:48-submit.csv
21-11-01_00:44:44-submit.csv
21-11-01_00:55:26-submit.csv
21-11-01_03:48:57-submit.csv
21-11-01_08:31:35-submit.csv
21-11-02_10:02:28-submit_2dmax.csv
21-11-02_12:46:31-submit_2dmax.csv
21-11-02_12:46:38-submit_2dmax_v002.csv
21-11-02_13:41:03-submit_2dmax_v003.csv
21-11-02_13:41:27-submit_2dmax_v002.csv
21-11-02_15:28:50-submit_2dmax_v004.csv
21-11-02_16:20:30-submit_2dmax_v004.csv
21-11-02_16:57:07-submit_2dmax_v005.csv
21-11-02_17:08:05-submit_2dmax_v007.csv
21-11-02_17:09:56-submit_2dmax_v008.csv
21-11-02_17:21:52-submit_2dmax_v009.csv
21-11-02_17:35:33-submit_2dmax_v010.csv
21-11-02_17:41:00-submit_2dmax_v011.csv
21-11-02_23:29:54-submit_lstm_v001.csv
21-11-02_23:34:53-submit_lstm_v002.csv
21-11-02_23:36:46-submit_lstm_v003.csv
21-11-02_23:37:45-008_th.csv
21-11-02_23:40:31-submit_lstm_v004.csv
21-11-02_23:52:43-submittttt.csv
21-11-02_23:54:27-submittttt.csv


In [99]:
# shutil.copy('./008_th.csv', './submission/submit/')

'./submission/submit/008_th.csv'

In [247]:
shutil.copy('./submittttt.csv', './submission/submit/')

'./submission/submit/submittttt.csv'

In [203]:
df = pd.read_csv('./submit_lstm_v004.csv')
list(df['answer']).count('Wake'),\
list(df['answer']).count('Light Sleep'),\
list(df['answer']).count('Deep Sleep')

(7883, 17174, 7834)

In [204]:
df = df.replace({"answer": {"Deep Sleep":0, "Light Sleep":1, "Wake":2}})

In [205]:
df2 = pd.read_csv('.//008_th.csv')

In [206]:
list(df2['answer']).count('Wake'),\
list(df2['answer']).count('Light Sleep'),\
list(df2['answer']).count('Deep Sleep')

(7530, 22189, 3172)

In [207]:
df2 = df2.replace({"answer": {"Deep Sleep":0, "Light Sleep":1, "Wake":2}})

In [208]:
label = ((np.array(df['answer'])+np.array(df2['answer']))/2).astype('int')

In [209]:
len(label)

32891

In [210]:
df['answer'] = label

In [212]:
df.to_csv('./submittttt.csv', index=False)

In [211]:
len(df)

32891

In [143]:
list(label).count(0)

7976

In [97]:
X.iloc[[922+856+803+765+855+781+933+834+748+786+881+775+957+830+782+838+694+748+954+764+845+832+858+788+853+839+820+852+954+715+868+657+727+870+813+728+757+838+834+933]]

,Case,Epoch,Epoch_Count,Frame_Count,Frame_Rate,Frames,answer
32887,A2021-EM-01-0163,933,935,139323,4.972,"[139166, 139315]",1


In [38]:
0005 - 922
0006 - 856
0020 - 803
0028 - 765
0081 - 855
0085 - 781
0090 - 933
0099 - 834
0110 - 743  ------>   748
0113 - 787  ------>   786
0117 - 881
0119 - 775
0182 - 957
0185 - 830
0186 - 782
0187 - 838
0005 - 694
0011 - 748
0014 - 954
0015 - 764
0021 - 845
0025 - 832
0028 - 858
0047 - 788
0053 - 853
0057 - 839
0058 - 820
0078 - 853  ------>   852
0082 - 954
0087 - 715
0101 - 868
0102 - 658  ------>   657
0110 - 727
0111 - 870
0112 - 813
0120 - 728
0127 - 757
0129 - 838
0162 - 834
0163 - 934


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-38-f1dd4f853267>, line 1)